# RDFIA : Notebook

## Fonctions utilitaires

In [1]:
import argparse
import time

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim.lr_scheduler
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
plt.ion()
import numpy as np

datasets.CIFAR10.url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
PRINT_INTERVAL = 50
CUDA = True
cudnn.benchmark = True

On définit la fonction d'accuracy top-k.

In [2]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

Classe pour calculer et stocker la valeur moyenne.

In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, keep_all=False):
        self.reset()
        self.data = None
        if keep_all:
            self.data = []

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        if self.data is not None:
            self.data.append(val)
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

Classe pour plot la loss de train par batch.

In [4]:
class TrainLossPlot(object):
    def __init__(self):
        self.loss_train = []
        self.fig = plt.figure()

    def update(self, loss_train):
        self.loss_train.append(loss_train)

    def plot(self):
        plt.figure(self.fig.number)
        plt.clf()
        plt.plot(np.array(self.loss_train))
        plt.title("Train loss / batch")
        plt.xlabel("Batch")
        plt.ylabel("Loss")
        plt.show()
        plt.draw_all()
        plt.pause(1e-3)

Classe pour plot l'accuracy et la loss train/test par epoch

In [5]:
class AccLossPlot(object):
    def __init__(self):
        self.loss_train = []
        self.loss_test = []
        self.acc_train = []
        self.acc_test = []
        self.fig = plt.figure()

    def update(self, loss_train, loss_test, acc_train, acc_test):
        self.loss_train.append(loss_train)
        self.loss_test.append(loss_test)
        self.acc_train.append(acc_train)
        self.acc_test.append(acc_test)
        plt.figure(self.fig.number)
        plt.clf()
        plt.subplot(1,2,1)
        plt.plot(np.array(self.acc_train), label="acc. train")
        plt.plot(np.array(self.acc_test), label="acc. test")
        plt.title("Accuracy / epoch")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.subplot(1,2,2)
        plt.plot(np.array(self.loss_train), label="loss train")
        plt.plot(np.array(self.loss_test), label="loss test")
        plt.title("Loss / epoch")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()
        plt.draw_all()
        plt.pause(1e-3)

## Modèles

Les imports pour les modèles.

In [6]:
import torch.nn as nn
import torchvision.models as models

### CNN similaire à VGG-16 (from scratch)

On implémente dans le réseau des couches de batch normalization ainsi qu'un dropout pour améliorer la stabilité du réseau et sa capacité de généralisation.

In [7]:
class ConvNet(nn.Module):
    """
    Cette classe contient la structure du réseau de neurones
    """

    def __init__(self, w_dropout=False, w_batchnorm=False):
        super(ConvNet, self).__init__()
        # Couches de convolution et pooling
        if w_batchnorm:
            self.features = nn.Sequential(
                nn.Conv2d(3, 32, (5, 5), stride=1, padding=2),
                nn.BatchNorm2d(32),
                nn.ReLU(),
                nn.MaxPool2d((2, 2), stride=2, padding=0),
                nn.Conv2d(32, 64, (5, 5), stride=1, padding=2),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d((2, 2), stride=2, padding=0),
                nn.Conv2d(64, 64, (5, 5), stride=1, padding=2),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d((2, 2), stride=2, padding=0, ceil_mode=True),
            )
        else:
            self.features = nn.Sequential(
                nn.Conv2d(3, 32, (5, 5), stride=1, padding=2),
                nn.ReLU(),
                nn.MaxPool2d((2, 2), stride=2, padding=0),
                nn.Conv2d(32, 64, (5, 5), stride=1, padding=2),
                nn.ReLU(),
                nn.MaxPool2d((2, 2), stride=2, padding=0),
                nn.Conv2d(64, 64, (5, 5), stride=1, padding=2),
                nn.ReLU(),
                nn.MaxPool2d((2, 2), stride=2, padding=0, ceil_mode=True)
            )
        # Couches fully connected
        if w_dropout:
            self.classifier = nn.Sequential(
                nn.Linear(64*4*4, 1000),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(1000, 10)
            )
        else:
            self.classifier = nn.Sequential(
                nn.Linear(64*4*4, 1000),
                nn.ReLU(),
                nn.Linear(1000, 10)
            )


    def forward(self, input):
        bsize = input.size(0) # Batch size
        output = self.features(input) # We compute the features
        output = output.view(bsize, -1) # Flattening
        output = self.classifier(output) # We compute the fully connected
        return output

### Instanciation des modèles

In [8]:
cnn = ConvNet()
cnn_bn = ConvNet(w_batchnorm=True)
cnn_dropout = ConvNet(w_dropout=True)
cnn_both = ConvNet(w_batchnorm=True, w_dropout=True)

## Main

### Transformations des données et chargement jeu de données

Ici on va faire de la data augmentation sur nos données afin d'augmenter les capacités de généralisation de notre réseau : un randomcrop et un random flip horizontal. De plus on va normaliser nos exemples.

In [9]:
# General parameters
BATCH_SIZE = 128
PATH = '/tmp/datasets/cifar'

compose_v1 = transforms.Compose([
                transforms.RandomCrop(28),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.491, 0.482, 0.447), (0.202, 0.199, 0.201))
                ])

compose_eval = transforms.Compose([
                transforms.RandomCrop(28),
                transforms.ToTensor(),
                transforms.Normalize((0.491, 0.482, 0.447), (0.202, 0.199, 0.201))
                ])

# Train et val loader pour la composition v1
train_dataset_v1 = datasets.CIFAR10(PATH, train=True, download=True,
                    transform=compose_v1)
eval_dataset_v1 = datasets.CIFAR10(PATH, train=False, download=True,
                    transform=compose_eval)

train_loader_v1 = torch.utils.data.DataLoader(train_dataset_v1,
                    batch_size=BATCH_SIZE, shuffle=True, pin_memory=CUDA, num_workers=2)
eval_loader_v1 = torch.utils.data.DataLoader(eval_dataset_v1,
                    batch_size=BATCH_SIZE, shuffle=True, pin_memory=CUDA, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


### Epochs

Définition d'une epoch

In [10]:
def epoch(data, model, criterion, optimizer=None):
    """
    Fait une passe (appelée epoch en anglais) sur les données `data` avec le
    modèle `model`. Evalue `criterion` comme loss.
    Si `optimizer` est fourni, effectue une epoch d'apprentissage en utilisant
    l'optimiseur donné, sinon, effectue une epoch d'évaluation (pas de backward)
    du modèle.
    """

    # indique si le modele est en mode eval ou train (certaines couches se
    # comportent différemment en train et en eval)
    model.eval() if optimizer is None else model.train()

    # objets pour stocker les moyennes des metriques
    avg_loss = AverageMeter()
    avg_top1_acc = AverageMeter()
    avg_top5_acc = AverageMeter()
    avg_batch_time = AverageMeter()
    global loss_plot

    # on itere sur les batchs du dataset
    tic = time.time()
    for i, (input, target) in enumerate(data):

        if CUDA: # si on fait du GPU, passage en CUDA
            input = input.cuda()
            target = target.cuda()

        # forward
        output = model(Variable(input))
        loss = criterion(output, Variable(target))

        # backward si on est en "train"
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # calcul des metriques
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        batch_time = time.time() - tic
        tic = time.time()

        # mise a jour des moyennes
        avg_loss.update(loss.data[0])
        avg_top1_acc.update(prec1[0])
        avg_top5_acc.update(prec5[0])
        avg_batch_time.update(batch_time)
        if optimizer:
            loss_plot.update(avg_loss.val)
            
        # affichage des infos
        if i % PRINT_INTERVAL == 0:
            print('[{0:s} Batch {1:03d}/{2:03d}]\t'
                  'Time {batch_time.val:.3f}s ({batch_time.avg:.3f}s)\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:5.1f} ({top1.avg:5.1f})\t'
                  .format(
                   "EVAL" if optimizer is None else "TRAIN", i, len(data), batch_time=avg_batch_time, loss=avg_loss,
                   top1=avg_top1_acc))
            if optimizer:
                loss_plot.plot()

    # Affichage des infos sur l'epoch
    print('\n===============> Total time {batch_time:d}s\t'
          'Avg loss {loss.avg:.4f}\t'
          'Avg Prec@1 {top1.avg:5.2f} %\t'
          'Avg Prec@5 {top5.avg:5.2f} %\n'.format(
           batch_time=int(avg_batch_time.sum), loss=avg_loss,
           top1=avg_top1_acc, top5=avg_top5_acc))

    return avg_top1_acc, avg_top5_acc, avg_loss

In [11]:
# init plots
plot = AccLossPlot()
global loss_plot
loss_plot = TrainLossPlot()

# Boucle sur les epochs

modeles=[cnn, cnn_bn, cnn_dropout, cnn_both]
for model in modeles:
    EPOCHS = 100
    # Learning rates / momentums
    lr_sgd = 0.1
    lr_adam = 0.0001
    momentum_sgd = 0.9
    # Optimizers
    optimizer_sgd = torch.optim.SGD(model.parameters(), lr_sgd, momentum_sgd)
    optimizer_adam = torch.optim.Adam(model.parameters(), lr_adam)
    # Scheduler
    lr_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_sgd, gamma=0.95)

    # Criterion
    criterion = nn.CrossEntropyLoss()
    
    # Cuda
    if CUDA:
        model.cuda()
        criterion = criterion.cuda()

    print("Compose v1 avec Adam")
    for i in range(EPOCHS):
        print("=================\n=== EPOCH "+str(i+1)+" =====\n=================\n")
        # Phase de train
        top1_acc, avg_top5_acc, loss = epoch(train_loader_v1, model, criterion, optimizer_adam)
        optimizer_adam.step()
        # Phase d'evaluation
        top1_acc_test, top5_acc_test, loss_test = epoch(eval_loader_v1, model, criterion)
        # plot
        print(top1_acc_test)
        plot.update(loss.avg, loss_test.avg, top1_acc.avg, top1_acc_test.avg)

Compose v2 avec Adam
=== EPOCH 1 =====



C:\Users\Sebastien\Anaconda3\envs\Deep Learning\lib\site-packages\ipykernel_launcher.py:45: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



===============> Total time 6s	Avg loss 1.7622	Avg Prec@1 35.26 %	Avg Prec@5 85.00 %


===============> Total time 1s	Avg loss 1.5624	Avg Prec@1 42.90 %	Avg Prec@5 90.87 %

=== EPOCH 2 =====


===============> Total time 5s	Avg loss 1.4791	Avg Prec@1 46.58 %	Avg Prec@5 92.13 %


===============> Total time 1s	Avg loss 1.3938	Avg Prec@1 49.81 %	Avg Prec@5 93.05 %

=== EPOCH 3 =====


===============> Total time 5s	Avg loss 1.3580	Avg Prec@1 51.62 %	Avg Prec@5 93.77 %


===============> Total time 1s	Avg loss 1.2989	Avg Prec@1 53.31 %	Avg Prec@5 94.28 %

=== EPOCH 4 =====


===============> Total time 5s	Avg loss 1.2685	Avg Prec@1 54.99 %	Avg Prec@5 94.64 %


===============> Total time 1s	Avg loss 1.2350	Avg Prec@1 56.46 %	Avg Prec@5 94.82 %

=== EPOCH 5 =====


===============> Total time 5s	Avg loss 1.2066	Avg Prec@1 57.31 %	Avg Prec@5 95.17 %


===============> Total time 1s	Avg loss 1.1745	Avg Prec@1 58.10 %	Avg Prec@5 95.51 %

=== EPOCH 6 =====


===============> Total time 5s	Avg


===============> Total time 5s	Avg loss 0.6380	Avg Prec@1 77.62 %	Avg Prec@5 98.76 %


===============> Total time 1s	Avg loss 0.7602	Avg Prec@1 73.95 %	Avg Prec@5 98.02 %

=== EPOCH 31 =====


===============> Total time 5s	Avg loss 0.6218	Avg Prec@1 78.31 %	Avg Prec@5 98.81 %


===============> Total time 1s	Avg loss 0.7501	Avg Prec@1 74.14 %	Avg Prec@5 98.35 %

=== EPOCH 32 =====


===============> Total time 5s	Avg loss 0.6135	Avg Prec@1 78.70 %	Avg Prec@5 98.80 %


===============> Total time 1s	Avg loss 0.7406	Avg Prec@1 74.83 %	Avg Prec@5 98.41 %

=== EPOCH 33 =====


===============> Total time 5s	Avg loss 0.6056	Avg Prec@1 78.80 %	Avg Prec@5 98.83 %


===============> Total time 1s	Avg loss 0.7157	Avg Prec@1 75.19 %	Avg Prec@5 98.49 %

=== EPOCH 34 =====


===============> Total time 5s	Avg loss 0.5938	Avg Prec@1 79.07 %	Avg Prec@5 98.88 %


===============> Total time 1s	Avg loss 0.7216	Avg Prec@1 75.59 %	Avg Prec@5 98.49 %

=== EPOCH 35 =====


===============> Total time 5


===============> Total time 5s	Avg loss 0.3931	Avg Prec@1 86.29 %	Avg Prec@5 99.59 %


===============> Total time 1s	Avg loss 0.6857	Avg Prec@1 77.35 %	Avg Prec@5 98.58 %

=== EPOCH 60 =====


===============> Total time 5s	Avg loss 0.3846	Avg Prec@1 86.61 %	Avg Prec@5 99.57 %


===============> Total time 1s	Avg loss 0.6586	Avg Prec@1 77.96 %	Avg Prec@5 98.57 %

=== EPOCH 61 =====


===============> Total time 5s	Avg loss 0.3850	Avg Prec@1 86.75 %	Avg Prec@5 99.60 %


===============> Total time 1s	Avg loss 0.6619	Avg Prec@1 78.18 %	Avg Prec@5 98.75 %

=== EPOCH 62 =====


===============> Total time 5s	Avg loss 0.3729	Avg Prec@1 86.88 %	Avg Prec@5 99.65 %


===============> Total time 1s	Avg loss 0.6833	Avg Prec@1 77.57 %	Avg Prec@5 98.78 %

=== EPOCH 63 =====


===============> Total time 5s	Avg loss 0.3705	Avg Prec@1 87.01 %	Avg Prec@5 99.65 %


===============> Total time 1s	Avg loss 0.6485	Avg Prec@1 78.37 %	Avg Prec@5 98.80 %

=== EPOCH 64 =====


===============> Total time 5

...


===============> Total time 5s	Avg loss 0.6322	Avg Prec@1 77.95 %	Avg Prec@5 98.75 %


===============> Total time 1s	Avg loss 0.6809	Avg Prec@1 76.32 %	Avg Prec@5 98.49 %

=== EPOCH 21 =====


===============> Total time 5s	Avg loss 0.6191	Avg Prec@1 78.40 %	Avg Prec@5 98.74 %


===============> Total time 1s	Avg loss 0.6842	Avg Prec@1 76.36 %	Avg Prec@5 98.47 %

=== EPOCH 22 =====


===============> Total time 5s	Avg loss 0.6085	Avg Prec@1 78.90 %	Avg Prec@5 98.87 %


===============> Total time 1s	Avg loss 0.7375	Avg Prec@1 74.39 %	Avg Prec@5 98.14 %

=== EPOCH 23 =====


===============> Total time 5s	Avg loss 0.6013	Avg Prec@1 79.04 %	Avg Prec@5 98.87 %


===============> Total time 1s	Avg loss 0.6599	Avg Prec@1 76.72 %	Avg Prec@5 98.51 %

=== EPOCH 24 =====


===============> Total time 5s	Avg loss 0.5922	Avg Prec@1 79.34 %	Avg Prec@5 98.98 %


===============> Total time 1s	Avg loss 0.6867	Avg Prec@1 76.17 %	Avg Prec@5 98.44 %

=== EPOCH 25 =====


===============> Total time 5


===============> Total time 5s	Avg loss 0.4274	Avg Prec@1 85.09 %	Avg Prec@5 99.46 %


===============> Total time 1s	Avg loss 0.5757	Avg Prec@1 80.62 %	Avg Prec@5 99.14 %

=== EPOCH 50 =====


===============> Total time 5s	Avg loss 0.4228	Avg Prec@1 85.19 %	Avg Prec@5 99.48 %


===============> Total time 1s	Avg loss 0.6068	Avg Prec@1 79.58 %	Avg Prec@5 98.80 %

=== EPOCH 51 =====


===============> Total time 5s	Avg loss 0.4182	Avg Prec@1 85.30 %	Avg Prec@5 99.50 %


===============> Total time 1s	Avg loss 0.5824	Avg Prec@1 80.46 %	Avg Prec@5 99.00 %

=== EPOCH 52 =====


===============> Total time 5s	Avg loss 0.4188	Avg Prec@1 85.39 %	Avg Prec@5 99.49 %


===============> Total time 1s	Avg loss 0.5672	Avg Prec@1 80.58 %	Avg Prec@5 98.96 %

=== EPOCH 53 =====


===============> Total time 5s	Avg loss 0.4086	Avg Prec@1 85.63 %	Avg Prec@5 99.54 %


===============> Total time 1s	Avg loss 0.5883	Avg Prec@1 80.25 %	Avg Prec@5 98.80 %

=== EPOCH 54 =====


===============> Total time 5


===============> Total time 5s	Avg loss 0.3264	Avg Prec@1 88.36 %	Avg Prec@5 99.71 %


===============> Total time 1s	Avg loss 0.5501	Avg Prec@1 81.76 %	Avg Prec@5 99.15 %

=== EPOCH 79 =====


===============> Total time 5s	Avg loss 0.3220	Avg Prec@1 88.79 %	Avg Prec@5 99.74 %


===============> Total time 1s	Avg loss 0.5620	Avg Prec@1 80.94 %	Avg Prec@5 99.20 %

=== EPOCH 80 =====


===============> Total time 5s	Avg loss 0.3213	Avg Prec@1 88.61 %	Avg Prec@5 99.72 %


===============> Total time 1s	Avg loss 0.5405	Avg Prec@1 81.89 %	Avg Prec@5 99.20 %

=== EPOCH 81 =====


===============> Total time 5s	Avg loss 0.3122	Avg Prec@1 89.20 %	Avg Prec@5 99.75 %


===============> Total time 1s	Avg loss 0.5625	Avg Prec@1 81.55 %	Avg Prec@5 99.15 %

=== EPOCH 82 =====


===============> Total time 5s	Avg loss 0.3112	Avg Prec@1 89.02 %	Avg Prec@5 99.73 %


===============> Total time 1s	Avg loss 0.5427	Avg Prec@1 81.96 %	Avg Prec@5 99.25 %

=== EPOCH 83 =====


===============> Total time 5

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

On se rend compte de l'efficacité de nos mesures pour augmenter la capacité de généralisation de notre réseau.